![](https://i.imgur.com/8qAo0Hu.png)
![](https://i.imgur.com/i784w5j.png)

**不能使用現成套件，只能使用numpy、scipy以及pandas。  
( Standard library可以 )  
(numpy.linalg.lstsq是不可以用的!!!)  **

In [58]:
import csv 
import numpy as np
from numpy.linalg import inv
import random
import math
import sys

## Training set
![](https://i.imgur.com/n6XjsIC.png) 
features 總共18種  
Each line represents one day  
Take 1-9 to predict 10, 2-10 to predict 11 .... , 14-23 to predict 24

In [59]:
data = []
# 每一個維度儲存一種污染物的資訊
for i in range(18):
    data.append([])

n_row = 0
text = open('data/train.csv', 'r', encoding='big5') 
row = csv.reader(text , delimiter=",")
for r in row:
    # 第0列沒有資訊
    if n_row != 0:
        # 每一列只有第3-27格有值(1天內24小時的數值)
        # data shape == (18, 5760) 12個月*20天*24小時
        for i in range(3,27):
            if r[i] != "NR":
                data[(n_row-1)%18].append(float(r[i]))
            else:
                data[(n_row-1)%18].append(float(0))
    n_row = n_row+1
text.close()

x = []
y = []
# 每 12 個月
for i in range(12):
    # 一個月取連續10小時的data可以有471筆 
    # 20 * 24 - 9 = 471
    for j in range(471):
        x.append([])
        # 18種污染物
        for t in range(18):
            # 連續9小時
            for s in range(9):
                x[471*i+j].append(data[t][480*i+j+s] )
        y.append(data[9][480*i+j+9])
x = np.array(x)
y = np.array(y)


# Add square term
# If your model is 
# y = b + w(1,pm)*x(1,pm) + w(1,pm)*(x(1,pm)**2) + w(2,pm)*x(2,pm) + w(2,pm)*(x(2,pm)**2) ....
# x = np.concatenate((x,x**2), axis=1)

# add bias
x = np.concatenate((np.ones((x.shape[0],1)),x), axis=1)

## validation set

In [60]:
# Set random seed
np.random.seed(0)
x_idx = np.arange(x.shape[0])
train_size = int(x.shape[0]*0.7)
training_idx = np.random.choice(x_idx, size=train_size, replace=False)
testing_idx = np.setdiff1d(x_idx, training_idx)
X_train, y_train, X_test, y_test = x[training_idx], y[training_idx], x[testing_idx], y[testing_idx]

![](https://i.imgur.com/rgf9RBd.png)

## Loss function
$\begin{equation}
\large{L(w,b) = \frac{1}{2}\sum_{n=0}^{N}(\hat{y}^n - (b + w \cdot x^n))^2} \\
\large{\frac{\partial L}{\partial w} = \sum_{n=0}^{N}(\hat{y}^n - (b + w \cdot x^n))}(-x^n) \\
\large{\frac{\partial L}{\partial b} = \sum_{n=0}^{N}(\hat{y}^n - (b + w \cdot x^n))}(-1)\\
\end{equation}$
## Adagrad
![](https://i.imgur.com/FEv1XCm.png)
![](https://i.imgur.com/0QNEqUe.png)

In [61]:
w = np.zeros(X_train.shape[1])
l_rate = 10
repeat = 10000


# use close form to check whether ur gradient descent is good
# however, this cannot be used in hw1.sh 
w_close_form = np.matmul(np.matmul(inv(np.matmul(x.transpose(),x)),x.transpose()),y)

s_gra = np.zeros(len(x[0]))

for i in range(repeat+1):
    hypo = np.dot(X_train, w) 
    loss = hypo - y_train # Loss function
    gra = np.dot(X_train.T,loss)
    s_gra += gra**2
    ada = np.sqrt(s_gra)
    w = w - l_rate * gra/ada # Adagrad
    if  not i % 500:
        cost = np.sum(loss**2) / X_train.shape[0]
        cost_a  = math.sqrt(cost)
        print ('iteration: {} / {} | Cost:{:.4f}  '.format( i, repeat,cost_a))

iteration: 0 / 10000 | Cost:26.8878  
iteration: 500 / 10000 | Cost:7.6084  
iteration: 1000 / 10000 | Cost:7.0088  
iteration: 1500 / 10000 | Cost:6.7119  
iteration: 2000 / 10000 | Cost:6.5260  
iteration: 2500 / 10000 | Cost:6.3971  
iteration: 3000 / 10000 | Cost:6.3018  
iteration: 3500 / 10000 | Cost:6.2280  
iteration: 4000 / 10000 | Cost:6.1691  
iteration: 4500 / 10000 | Cost:6.1207  
iteration: 5000 / 10000 | Cost:6.0802  
iteration: 5500 / 10000 | Cost:6.0458  
iteration: 6000 / 10000 | Cost:6.0161  
iteration: 6500 / 10000 | Cost:5.9903  
iteration: 7000 / 10000 | Cost:5.9677  
iteration: 7500 / 10000 | Cost:5.9476  
iteration: 8000 / 10000 | Cost:5.9297  
iteration: 8500 / 10000 | Cost:5.9136  
iteration: 9000 / 10000 | Cost:5.8991  
iteration: 9500 / 10000 | Cost:5.8859  
iteration: 10000 / 10000 | Cost:5.8739  


In [62]:
# predict
hypo = np.dot(X_test, w)
loss = hypo - y_test
cost_a = math.sqrt(np.sum(loss**2)/X_train.shape[0])
print('Test set cost: {:.4f}'.format(cost_a))
print('Predict: {}'.format(hypo))
print('Label: {}'.format(y_test))

Test set cost: 3.9014
Predict: [21.02803603 42.02269111 25.96815356 ...  3.37310476 35.88128301
 21.64747507]
Label: [30. 42. 22. ...  7. 49. 17.]


## 標準化
![](https://i.imgur.com/3cfTZ7W.png)

In [63]:
x_std = []
means = []
stds = []
for i in range(1, x.shape[1]):
    mean = np.mean(x[:,i])
    std = np.std(x[:,i])
    x_std.append(((x[:, i] - mean)/std))
    stds.append(std)
    means.append(mean)
x_std = np.concatenate((np.ones((1, x.shape[0])), x_std)).T
X_train_std, X_test_std = x_std[training_idx], x_std[testing_idx]

w_std = np.zeros(X_train.shape[1])
l_rate = 10

s_gra = np.zeros(len(x[0]))

for i in range(repeat+1):
    hypo = np.dot(X_train_std, w_std) 
    loss = hypo - y_train # Loss function
    gra = np.dot(X_train_std.T,loss)
    s_gra += gra**2
    ada = np.sqrt(s_gra)
    w_std = w_std - l_rate * gra/ada # Adagrad
    if  not i % 500:
        cost = np.sum(loss**2) / X_train.shape[0]
        cost_a  = math.sqrt(cost)
        print ('iteration: {} / {} | Cost:{:.4f}  '.format( i, repeat,cost_a))

iteration: 0 / 10000 | Cost:26.8878  
iteration: 500 / 10000 | Cost:6.0724  
iteration: 1000 / 10000 | Cost:5.8415  
iteration: 1500 / 10000 | Cost:5.7586  
iteration: 2000 / 10000 | Cost:5.7187  
iteration: 2500 / 10000 | Cost:5.6971  
iteration: 3000 / 10000 | Cost:5.6848  
iteration: 3500 / 10000 | Cost:5.6776  
iteration: 4000 / 10000 | Cost:5.6732  
iteration: 4500 / 10000 | Cost:5.6705  
iteration: 5000 / 10000 | Cost:5.6688  
iteration: 5500 / 10000 | Cost:5.6677  
iteration: 6000 / 10000 | Cost:5.6669  
iteration: 6500 / 10000 | Cost:5.6663  
iteration: 7000 / 10000 | Cost:5.6659  
iteration: 7500 / 10000 | Cost:5.6655  
iteration: 8000 / 10000 | Cost:5.6652  
iteration: 8500 / 10000 | Cost:5.6650  
iteration: 9000 / 10000 | Cost:5.6647  
iteration: 9500 / 10000 | Cost:5.6645  
iteration: 10000 / 10000 | Cost:5.6643  


In [64]:
# predict with std
hypo = np.dot(X_test_std, w_std)
loss = hypo - y_test
cost_a = math.sqrt(np.sum(loss**2)/X_train.shape[0])
print('Test_std set cost: {:.4f}'.format(cost_a))
print('Predict: {}'.format(hypo))
print('Label: {}'.format(y_test))

Test_std set cost: 3.8352
Predict: [21.35042856 42.15397482 27.42859766 ...  3.83940181 37.79955571
 20.45109027]
Label: [30. 42. 22. ...  7. 49. 17.]


![](https://i.imgur.com/fFFsIkU.png)

In [65]:
test_x = []
n_row = 0
text = open('data/test.csv' ,"r")
row = csv.reader(text , delimiter= ",")

for r in row:
    if n_row %18 == 0:
        test_x.append([])
        for i in range(2,11):
            test_x[n_row//18].append(float(r[i]) )
    else :
        for i in range(2,11):
            if r[i] !="NR":
                test_x[n_row//18].append(float(r[i]))
            else:
                test_x[n_row//18].append(0)
    n_row = n_row+1
text.close()
test_x = np.array(test_x)

# Add square term
# If your model is 
# y = b + w(1,pm)*x(1,pm) + w(1,pm)*(x(1,pm)**2) + w(2,pm)*x(2,pm) + w(2,pm)*(x(2,pm)**2) ....
# test_x = np.concatenate((test_x,test_x**2), axis=1)

n_row = 0

ans_text = open('data/ans.csv', "r")
row = csv.reader(ans_text, delimiter=',')
ans = []
for i,r in enumerate(row):
    if i != 0:
        ans.append(r[1])
ans = np.array(ans).astype(np.float64)

test_x_std = []

for i, (mean, std) in enumerate(zip(means, stds)):
    test_x_std.append((test_x[:,i]-mean)/std)
test_x_std = np.concatenate((np.ones((1, test_x.shape[0])), test_x_std)).T
test_x = np.concatenate((np.ones((test_x.shape[0], 1)), test_x), axis=1)

In [66]:
# predict
hypo = np.dot(test_x, w)
loss = hypo - ans
cost_a = math.sqrt(np.sum(loss**2)/test_x.shape[0])
print('test_x set cost: {:.4f}'.format(cost_a))

# predict with std
hypo = np.dot(test_x_std, w_std)
loss = hypo - ans
cost_a = math.sqrt(np.sum(loss**2)/test_x_std.shape[0])
print('test_x_std set cost: {:.4f}'.format(cost_a))

test_x set cost: 6.9051
test_x_std set cost: 6.6682


In [67]:
# predict with close_form_solution
hypo = np.dot(test_x, w_close_form)
loss = hypo - ans
cost_a = math.sqrt(np.sum(loss**2)/test_x.shape[0])
print('test_x set cost: {:.4f}'.format(cost_a))

test_x set cost: 6.5700
